In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

In [3]:
print(azureml.core.VERSION)

1.23.0


In [12]:
ws.write_config()

NameError: name 'ws' is not defined

In [4]:
ws = Workspace.create(name='apartment',
                     subscription_id='313558ae-62b7-424d-a6b0-343c89a678b7',
                     resource_group='linear_regression_prediction', 
                     create_resource_group=True,
                     location='eastus'
                     )

WorkspaceException: WorkspaceException:
	Message: Workspace with name 'apartment' already exists under resource group with name 'linear_regression_prediction'.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Workspace with name 'apartment' already exists under resource group with name 'linear_regression_prediction'."
    }
}

In [ ]:
ws.write_config()

In [5]:
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/313558ae-62b7-424d-a6b0-343c89a678b7/resourceGroups/linear_regression_prediction/providers/Microsoft.MachineLearningServices/workspaces/apartment',
 'name': 'apartment',
 'identity': {'principal_id': '8581e750-2332-4fe6-ad4f-7d1daeab698f',
  'tenant_id': '1b6d1f48-8893-4889-bdbf-1fb841bcae46',
  'type': 'SystemAssigned'},
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Basic',
 'workspaceid': '78239986-ddcb-4f2e-8928-54fe8c41e60b',
 'sdkTelemetryAppInsightsKey': 'f5784ccd-178d-4ecc-9998-b05841b44ae9',
 'description': '',
 'friendlyName': 'apartment',
 'creationTime': '2021-02-27T17:47:17.3100475+00:00',
 'containerRegistry': '/subscriptions/313558ae-62b7-424d-a6b0-343c89a678b7/resourceGroups/linear_regression_prediction/providers/Microsoft.ContainerRegistry/registries/78239986ddcb4f2e892854fe8c41e60b',
 'keyVault': '/subscriptions/313558ae-62b7-424d-a6b0-343c89a678b7/resourcegroups/linear_regression_prediction/providers/mi

In [20]:
exp = Experiment(workspace=ws, name='apartamentexp')

In [25]:
run = exp.start_logging()
run.log("Experiment start time",str(datetime.datetime.now()))

In [27]:
data_exp = pd.read_csv('apartmentComplexData.txt', sep=",", header=None)

In [28]:
data_exp.columns = ['column1','column2','complexAge','totalRooms','totalBedrooms','complexInhabitants','apartmentsNr','column8','medianCompexValue']


In [29]:
x = data_exp[['column1','column2','complexAge','totalRooms','totalBedrooms','complexInhabitants','apartmentsNr','column8']]


In [30]:
y = data_exp['medianCompexValue']


In [31]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.40,random_state=42)


In [32]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(x_train,y_train)

LinearRegression()

In [33]:
filename = 'outputs/apartament_exp_model.pkl'
joblib.dump(lm,filename)

['outputs/apartament_exp_model.pkl']

In [41]:
filename = 'outputs/apartament_exp_model.pkl'
loaded_model = joblib.load(filename)
y = loaded_model.predict([[-122,37,41,880,129,322,126,8]])

In [42]:
print(y)

[426102.65557006]


In [43]:
run.log('Intercept :', lm.intercept_)
run.log('Slope:', lm.coef_[0])

In [44]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

In [45]:
print(run.get_portal_url())

https://ml.azure.com/experiments/apartamentexp/runs/e07fa37e-ce04-4582-ac96-b70bbf90fe8a?wsid=/subscriptions/313558ae-62b7-424d-a6b0-343c89a678b7/resourcegroups/linear_regression_prediction/workspaces/apartment


In [46]:
model = Model.register(model_path="outputs/apartament_exp_model.pkl",
                      model_name = "apartament_exp_model",
                      tags = {"key":"1"},
                      description = "Salary Prediction",
                      workspace = ws)

Registering model apartament_exp_model


In [47]:
acicconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {"data":"apartament","method":"sklearn"},
                                               description = 'Predict apartament price')

In [52]:
apartamentenv = CondaDependencies()
apartamentenv.add_conda_package("scikit-learn")

with open("apartamentenv.yml","w") as f:
    f.write(apartamentenv.serialize_to_string())
with open("apartamentenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [51]:
%%writefile score.py
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
import pickle
import json
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model


def init():
    global model
    model_path = Model.get_model_path('apartment_model')
    model = joblib.load(model_path)

def run(raw_data):
	data = np.array(json.loads(raw_data)['data'])
	y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())




Overwriting score.py


In [60]:
image_config = ContainerImage.image_configuration(execution_script="score.py",
                                                 runtime="python",
                                                 conda_file = "apartamentenv.yml")

<ipython-input-60-529e0513b7d9>:1: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  image_config = ContainerImage.image_configuration(execution_script="score.py",


In [ ]:
services = Webservice.deploy_from_model(workspace=ws,
                                        name='apartament-svc',
                                        models = [model],
                                        image_config = image_config)

<ipython-input-66-9400d7182b1d>:1: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  services = Webservice.deploy_from_model(workspace=ws,


Creating image
Running.........................................................................

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


env = Environment.get(ws, "AzureML-Minimal").clone(apartamentenv)

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='D:\liniar regresion\lab\score.py',
                                    environment=apartamentenv)

In [6]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

In [8]:
env = Environment.get(ws, "AzureML-Minimal").clone(env_name)


NameError: name 'env_name' is not defined

In [83]:
model = Model.register(workspace = ws,
              model_path ="model/apartment_model.pkl",
              model_name = "apartment_model",
              tags = {"version": "1"},
              description = "Predict apartment price",
              )

Registering model apartment_model


In [84]:
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies

In [85]:
env = Environment('env')

In [86]:
cd = CondaDependencies.create(pip_packages=['pandas==1.1.5','numpy==1.18.1','azureml-defaults','joblib==1.0.1','seaborn==0.11.1','matplotlib==3.3.4'], conda_packages = ['scikit-learn==0.23.2'])
env.python.conda_dependencies = cd

In [115]:
env.register(workspace = ws)
print("Registered Environment")

Registered Environment


In [116]:
myenv = Environment.get(workspace=ws, name="env")

In [117]:
myenv.save_to_directory('./environ', overwrite=True)

In [118]:
aciconfig = AciWebservice.deploy_configuration(
            cpu_cores=1,
            memory_gb=1,
            tags={"data":"apartment model"},
            description='apartment model for prediction price',
            )

In [119]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

In [120]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)


In [121]:
inference_config = InferenceConfig(entry_script="score.py",environment=myenv)

In [48]:
%%writefile score.py
import os
import pandas as pd
import numpy as np

import joblib
import pickle
import json
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model


def init():
    global model
    model_path = Model.get_model_path("apartment_model")
    print("Model Path is  ", model_path)
    model = joblib.load(model_path)

def run(data):
    try:
       print(data)
       result = model.predict(data['data']) 
       return {'data' : result.tolist() , 'message' : "Successfully  predict price"}
    except Exception as e:
           error = str(e)
           return {'data' : error , 'message' : 'Failed to receive data'}



Overwriting score.py


In [140]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

In [58]:
inference_config = InferenceConfig(entry_script="D:\liniar regresion\lab\score.py", environment=myenv)

In [138]:
inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

In [114]:
cd = CondaDependencies.create(pip_packages=['pandas==1.1.5','numpy==1.18.1', 'azureml-defaults','joblib==0.17.0'], conda_packages = ['scikit-learn==0.23.2'])
env.python.conda_dependencies = cd

In [141]:
service = Model.deploy(workspace=ws,
                name='apartment-model',
                models=[model],
                inference_config=inference_config,
                deployment_config=aciconfig, 
                overwrite = True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://2848558a-3d7a-49b9-8b13-4d25eb0139fc.eastus.azurecontainer.io/score


In [112]:
from azureml.core import Webservice
service = Webservice(ws,'apartment-model')
print(service.get_logs())

2021-02-28T17:34:48,515826000+00:00 - rsyslog/run 
2021-02-28T17:34:48,517233200+00:00 - iot-server/run 
2021-02-28T17:34:48,526956800+00:00 - nginx/run 
2021-02-28T17:34:48,515829000+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_dd02f604ff8a8a7e3b4f719af529c10c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dd02f604ff8a8a7e3b4f719af529c10c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dd02f604ff8a8a7e3b4f719af529c10c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dd02f604ff8a8a7e3b4f719af529c10c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dd02f604ff8a8a7e3b4f719af529c10c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [146]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': [[-122,37,41,880,129,322,126,8]] } , headers = headers)
print(r.status_code)
print(r.json())

502


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [147]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': [[1,1,1,1]] } , headers = headers)
print(r.status_code)
print(r.json())

502


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [148]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {[0]: [[1,1,1,1]] } , headers = headers)
print(r.status_code)
print(r.json())

TypeError: unhashable type: 'list'

In [65]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': [[1,1,1,1]] } , headers = headers)
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to receive data'}


In [66]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'0': [[1,1,1,1]] } , headers = headers)
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to receive data'}


In [67]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': [1,1,1,1] } , headers = headers)
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to receive data'}


In [68]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': [0][1,1,1,1] } , headers = headers)
print(r.status_code)
print(r.json())

<>:3: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:3: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<ipython-input-68-90419b9403c2>:3: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  r = requests.post(url, data = {'data': [0][1,1,1,1] } , headers = headers)


TypeError: list indices must be integers or slices, not tuple

In [136]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': [1,1,1,1] } , headers = headers)
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to classify iris'}


In [75]:
!curl -X POST \
  -H 'Content-Type':'application/json' \
  -d '{"data":[[-122,37,41,880,129,322,126,8]]}' \
      2848558a-3d7a-49b9-8b13-4d25eb0139fc.eastus.azurecontainer.io/score


{"message": "Expects Content-Type to be application/json"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    41    0     0  100    41      0     41  0:00:01 --:--:--  0:00:01   187
100    99  100    58  100    41     58     41  0:00:01 --:--:--  0:00:01   372


In [77]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {[1,1,1,1] } , headers = headers)
print(r.status_code)
print(r.json())

TypeError: unhashable type: 'list'

In [135]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': [1,1,1,1] } , headers = headers)
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to classify iris'}


In [134]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': "1,1,1,,1" } , headers = headers)
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to classify iris'}


In [133]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': 1 } , headers = headers)
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to classify iris'}


In [132]:
import requests
headers = {'Content-Type':'application/json'}
r = requests.post(url, data = {'data': "1" } , headers = headers)
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to classify iris'}


In [131]:
import requests
headers = {'Content-Type':'application/json'}
data = {'data': "1"}
r = requests.post(url, data = data, headers = headers)
print(r.text)
print(r.status_code)
print(r.json())

{"data": "string indices must be integers", "message": "Failed to classify iris"}
200
{'data': 'string indices must be integers', 'message': 'Failed to classify iris'}


In [129]:
headers = {'Content-Type':'application/json'}
r = requests.post(url,data = {}, headers = headers)


In [130]:
print(r.status_code)
print(r.json())

200
{'data': 'string indices must be integers', 'message': 'Failed to classify iris'}


In [127]:
import requests
headers = {'Content-Type':'application/json'}
data = {"data": [1,2,2,3,3]}
r = requests.post(url, data = data, headers = headers)
print(r.text)
print(r.status_code)
print(r.json())

{"data": "string indices must be integers", "message": "Failed to classify iris"}
200
{'data': 'string indices must be integers', 'message': 'Failed to classify iris'}


In [143]:
import requests
headers = {'Content-Type':'application/json'}
data = {"data" : "1"}
r = requests.post(url, data = data, headers = headers)
print(r.text)
print(r.status_code)
print(r.json())

Expecting value: line 1 column 1 (char 0)
502


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [149]:
import requests
import json
headers = {'Content-Type':'application/json'}
test_sample = json.dumps({'data': [
    [-122,37,41,880,129,322,126,8]
]})

r = requests.post(url, data = test_sample, headers = headers)
print(r.text)
print(r.status_code)
print(r.json())

"[426102.65557006374]"
200
[426102.65557006374]
